Links:


https://python.langchain.com/docs/concepts/tools/

https://python.langchain.com/docs/how_to/custom_tools/

https://python.langchain.com/docs/integrations/tools/

## 2. **Objective**  
I want to enable an AI application to access my Gmail emails, have it check my inbox, and see if "XX" has sent me any messages.  
*(Because I might be eagerly awaiting their message)*

**Tip:** **Develop a personal assistant using the Gmail toolkit**

### **Step 1:**  
Set up your application interface in Google Cloud — `credential.json`

### **Step 2:**  
Generate development tokens based on the keys — `token.json`

### **Step 3:**  
Develop a Gmail App using the LangChain framework


# Generating the token

In [21]:
import os 

credential_path = os.path.dirname(os.getcwd())



In [8]:
'''Generate the token'''

from google_auth_oauthlib.flow import InstalledAppFlow

# read-only access to Gmail
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

secret_path = os.path.join(credential_path, "credentials.json")

def generate_token():
    flow = InstalledAppFlow.from_client_secrets_file(secret_path, SCOPES)
    creds = flow.run_local_server(port=0)
    with open("token.json", "w") as token_file:
        token_file.write(creds.to_json())
    print("token.json generated successfully!")

generate_token()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=483415711433-4d24q8dqhulhpv40erp0ll0oma50sevq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53231%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=lqe2xdLShgrxeC20oWmhX7uIO4shCB&access_type=offline
token.json generated successfully!


# Use Gmail in LangChain

In [8]:
# %pip install -qU "langchain-google-community[gmail]"


Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_google_community import GmailToolkit
from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
# 'https://www.googleapis.com/auth/gmail.readonly' cannot send, delete, or modify
# 'https://mail.google.com/' gives the full access
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://www.googleapis.com/auth/gmail.readonly/"],
    client_secrets_file=secret_path,
)
api_resource = build_resource_service(credentials=credentials)
toolkit = GmailToolkit(api_resource=api_resource)

tools = toolkit.get_tools()

# View available tools
for tool in tools:
    print(f"- {tool.name} \t {tool.description}")

- create_gmail_draft 	 Use this tool to create a draft email with the provided message fields.
- send_gmail_message 	 Use this tool to send email messages. The input is the message, recipients
- search_gmail 	 Use this tool to search for email messages or threads. The input must be a valid Gmail query. The output is a JSON list of the requested resource.
- get_gmail_message 	 Use this tool to fetch an email by message ID. Returns the thread ID, snippet, body, subject, and sender.
- get_gmail_thread 	 Use this tool to search for email messages. The input must be a valid Gmail query. The output is a JSON list of messages.


In [14]:
from langchain_google_community.gmail.search import GmailSearch
from langchain_google_community.gmail.utils import get_gmail_credentials, build_resource_service

# Set up credentials
creds = get_gmail_credentials(
    token_file="token.json",
    client_secrets_file=secret_path,
    scopes=["https://www.googleapis.com/auth/gmail.readonly"]
)

resource = build_resource_service(credentials=creds)

# Initialize the GmailSearch tool
search_tool = GmailSearch(api_resource=resource)

# Run a Gmail query 
results = search_tool.run("from:noreply@langchain.com label:unread")  

print(results)


[{'id': '1973dd119eee39b0', 'threadId': '1973dd119eee39b0', 'snippet': 'Prompt engineering is key to LLM application development because the quality of a prompt directly affects the LLM&#39;s output. With the right prompts, you can ensure your app delivers accurate,', 'body': 'Prompt engineering is key to LLM application development because the quality of a prompt directly affects the LLM’s output. With the right prompts, you can ensure your app delivers accurate, relevant, and reliable responses.\r\n\r\nLangSmith’s Playground is your go-to space for experimenting with different prompts and models on various inputs, allowing you to quickly see how changes affect the response.\r\n\r\nWatch this quick video to see the Playground in action: https://track.pstmrk.it/3ts/youtu.be%2FsuJU1VYzy50/Tpyp/Gc69AQ/AQ/6397310f-ffa7-44cf-8df4-48cad3c76335/1/7QFXOxeco5\r\n\r\nJump in and check out our Playground docs to get started! https://track.pstmrk.it/3ts/docs.smith.langchain.com%2Fprompt_engineeri